### This code extracts the desired variables from the ACS5 2018 Census on a Congressional District level to include in the interactive map

#### The desired variables are as follows:
##### Mexican pop

B03001_004E - Estimate!Total!Hispanic or Latino!!Mexican

__
##### Latino pop
B03001_003E - Estimate!!Total!!Hispanic or Latino 

_
##### Total population

B02001_001E - Estimate!!Total (Race) (Total population)
    

In [1]:
import tabpy

In [24]:
import pandas as pd
import us
import census 
from us import states 
from census import Census

In [25]:
#saving API key, opening session, and specifying variables

KEY = '0bde28441892c9a213ccb5782415c27f88130a35'
C = Census(key, year=2018)
CODES = {'B03001_004E': "Mexican Pop", 'B03001_003E': 'Latino Pop', 'B02001_001E' : 'Total Pop'}

#extracting information from census and loading into data frame

cong = C.acs5.state_congressional_district(list(CODES.keys()), "*", "*")
cong = pd.DataFrame(cong)

In [26]:
def format_district(row):
    '''
    This function formats congressional district into the format of the district shape files,
    given the current format provided by the census library
    '''
    if row == '00':
        return 'Congressional District (at Large)'
    elif row == '98':
        return 'Delegate District (at Large)'
    elif row[0] == '0':
        return 'Congressional District ' + row.replace('0', '')
    else:
        return 'Congressional District ' + row

In [27]:
#converting FIPS to state names

cong['Name'] = cong['state'].apply(lambda x: str(states.lookup(x)))

#Renaming code columns
cong.rename(columns=CODES, inplace=True)

#dropping obsolete congressional districts and Puerto Rico

cong = cong[cong['congressional district'] != 'ZZ']
cong= cong[cong['Name'] != 'Puerto Rico']

#string cleaning with function above

cong['Namelsad'] = cong['congressional district'].apply(format_district)

In [28]:
#for spreadsheet organization
cong.sort_values(by=['Name', 'congressional district'], inplace=True)

In [30]:
#selecting desired columns 
df = cong[['Mexican Pop', 'Latino Pop', 'Total Pop', 'Name', 'Namelsad']]

#export dataframe to excel
df.to_excel("./Data/census_data.xlsx")